# An Experiment with Embeddings, Vector Database, and Similarity Metrics

We query a dataset that has semantically similar and dissimilar sentences to the query sentence. Only known similar sentences should be returned in query results. We assign an accuracy score based on the actual versus expected results,

We conduct the experiment using the following tools:

Embeddings:
  - Hugging Face's 'all-MiniLM-L6-v2'
  - Open AI's "text-embedding-ada-002"

Embeddings are stored and indexed in Pinecone vector database.

Similarity functions:
  - Cosine
  - Dot product
  - Euclidean distance

## Helper functions

In [1]:
import pickle

# save object to file
def save(object, tofile):
    with open(tofile, 'wb') as fp:
        pickle.dump(object, fp)
    return

# load object from file
def load(fromfile):
    with open(fromfile, 'rb') as fp:
        object = pickle.load(fp)
    return object

## Load dataset and queries

In [4]:
datadir = 'data/'
datadir = 'sample_data/'
# load dataset and query sentences
docs = load(datadir+'docs.pickle')
print(f'docs[0]: {docs[0]}')
q1_sentence = load(datadir+'q1.pickle')
print(f'query sentence 1: {q1_sentence}')
q2_sentence = load(datadir+'q2.pickle')
print(f'query sentence 2: {q2_sentence}')

docs[0]: The car skidded to stop for the deer that stood frozen in the headlights of the car.
query sentence 1: The deer froze in the headlights of the car.
query sentence 2: Dream to solve world's problems.


## Embeddings with `all-MiniLM-L6-v2`

### Load saved embeddings

In [6]:
text_embedding_model = 'all-MiniLM-L6-v2'
docs_file = f'{datadir}docs-{text_embedding_model}.pickle'

try:
  doc_embeddings = load(docs_file)
  doc_embeddings_list = doc_embeddings.tolist()
  print('Dataset embeddings loaded.')
  print(f'shape: {doc_embeddings.shape}')

except e:
  print(f'Exception: {e}. Use Part 1 ntoebook to generate these embeddings.')


Dataset embeddings loaded.
shape: torch.Size([57, 384])


In [ ]:
q1_file = f'{datadir}q1-{text_embedding_model}.pickle'
q2_file = f'{datadir}q2-{text_embedding_model}.pickle'

try:
  q1_embeddings = load(q1_file)
  q2_embeddings = load(q2_file)
  print('Query embeddings loaded.')
  print(f'shape: {q1_embeddings.shape}')

except e:
  print(f'Exception: {e}. Use Part 1 ntoebook to generate these embeddings.')


Query embeddings loaded.
shape: torch.Size([384])


## Initialize the vector database

In [5]:
!pip3 install pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 14.9 MB/s eta 0:00:00


In [6]:
!pip3 install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [109]:
from dotenv import dotenv_values

#config = dotenv_values(".env")
#pineconse_api_key = config['PINECONE_API_KEY']
#pineconse_env = config['PINECONE_ENVIRONMENT']

In [8]:
import pinecone

pinecone.init(
    api_key=pineconse_api_key,
    environment=pineconse_env
)

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Indexes, similarity searches, and comparisons



In [9]:
# convenience functions to create and populate indexes in pinecone

pinecone_index = None
def create_index(index_to_create, dimension, metric):
  # only create index if it doesn't exist
  if index_to_create not in pinecone.list_indexes():
      pinecone.create_index(
          name=index_to_create,
          dimension=dimension,
          metric=metric
      )
  return

def delete_index(index_name):
  pinecone.delete_index(index_name)

def populate_index(index_name, upsert_docs):
  global pinecone_index
  pinecone_index = pinecone.Index(index_name)
  return pinecone_index.upsert(upsert_docs)

def query_index(q_embeddings):
  # submit query
  xc = pinecone_index.query(q_embeddings, top_k=7, include_metadata=True)
  print(*[docs[int(xc['matches'][i]['id'])] for i in range(len(xc['matches']))], sep='\n')
  return

In [103]:
docs_to_upsert = [(str(i), de.tolist()) for i, de in enumerate(doc_embeddings)]

### Cosine similarity search and results

In [67]:
index_name = 'minilm-cosine'
create_index(index_name, len(doc_embeddings_list[0]), 'cosine')
populate_index(index_name, docs_to_upsert)

{'upserted_count': 57}

In [70]:
query_index(q1_embeddings.tolist())

The car skidded and stopped for the frozen deer in its headlights.
The car skidded to stop for the deer that stood frozen in the headlights of the car.
The car skidded to a standstill for the deer that remained motionless in the car's headlights.
The car skidded and stopped to avoid the motionless deer illuminated by its headlights.
The vehicle slid and came to a halt in response to the deer's immobility under the car's headlights.
The scientific article explains what causes animals to freeze staring into the headlights of speeding vehicles causing many deaths and accidents every year.
As the car lost traction, it slid across the road and eventually halted abruptly, its brakes screeching, due to the presence of a motionless deer standing in the direct path of its headlights.


5 out of top 6 results are the expected semantically similar results. Acccuracy: 5/6 = 83%.



In [69]:
query_index(q2_embeddings.tolist())

The 16-year-old genius, having earned a PhD, contemplated global challenges and recognized science as the solution.
Having completed his doctorate at 16, the exceptionally gifted young prodigy pondered the difficulties confronting the world, intuitively perceiving science as the remedy.
Having finished his PhD at 16, the boy genius  contemplated the challenges the world faced, and intuited that science must be the solution.
The boy genius finished his PhD at 16 and believed science was the solution to the world's challenges.
After completing his PhD at the age of 16, the exceptionally talented young prodigy reflected on the global challenges and recognized science as the answer.
Finding sustainable energy solutions is crucial for a greener future.
Having successfully obtained his PhD at the remarkably young age of 16, the boy genius engaged in deep contemplation of the complex issues faced by the world. His intuition led him to firmly believe that science held the key to addressing the

5 out of top 6 results are the expected semantically similar results. Acccuracy: 5/6 = 83%.



In [72]:
delete_index('minilm-cosine')

### Euclidean Distance similarity search and results

In [104]:
index_name = 'minilm-euclidean'
create_index(index_name, len(doc_embeddings_list[0]), 'euclidean')
populate_index(index_name, docs_to_upsert)

{'upserted_count': 57}

In [105]:
query_index(q1_embeddings.tolist())

The car skidded and stopped for the frozen deer in its headlights.
The car skidded to stop for the deer that stood frozen in the headlights of the car.
The car skidded to a standstill for the deer that remained motionless in the car's headlights.
The car skidded and stopped to avoid the motionless deer illuminated by its headlights.
The vehicle slid and came to a halt in response to the deer's immobility under the car's headlights.
The scientific article explains what causes animals to freeze staring into the headlights of speeding vehicles causing many deaths and accidents every year.
As the car lost traction, it slid across the road and eventually halted abruptly, its brakes screeching, due to the presence of a motionless deer standing in the direct path of its headlights.


5 out of top 6 results are the expected semantically similar results. Acccuracy: 5/6 = 83%.



In [106]:
query_index(q2_embeddings.tolist())

The 16-year-old genius, having earned a PhD, contemplated global challenges and recognized science as the solution.
Having completed his doctorate at 16, the exceptionally gifted young prodigy pondered the difficulties confronting the world, intuitively perceiving science as the remedy.
Having finished his PhD at 16, the boy genius  contemplated the challenges the world faced, and intuited that science must be the solution.
The boy genius finished his PhD at 16 and believed science was the solution to the world's challenges.
After completing his PhD at the age of 16, the exceptionally talented young prodigy reflected on the global challenges and recognized science as the answer.
Finding sustainable energy solutions is crucial for a greener future.
Having successfully obtained his PhD at the remarkably young age of 16, the boy genius engaged in deep contemplation of the complex issues faced by the world. His intuition led him to firmly believe that science held the key to addressing the

5 out of top 6 results are the expected semantically similar results. Acccuracy: 5/6 = 83%.



In [107]:
delete_index('minilm-euclidean')

### Dot Product similarity search and results

In [94]:
index_name = 'minilm-dotproduct'
create_index(index_name, len(doc_embeddings_list[0]), 'dotproduct')
populate_index(index_name, docs_to_upsert)

{'upserted_count': 57}

In [95]:
query_index(q1_embeddings.tolist())

The car skidded and stopped for the frozen deer in its headlights.
The car skidded to stop for the deer that stood frozen in the headlights of the car.
The car skidded to a standstill for the deer that remained motionless in the car's headlights.
The car skidded and stopped to avoid the motionless deer illuminated by its headlights.
The vehicle slid and came to a halt in response to the deer's immobility under the car's headlights.
The scientific article explains what causes animals to freeze staring into the headlights of speeding vehicles causing many deaths and accidents every year.
As the car lost traction, it slid across the road and eventually halted abruptly, its brakes screeching, due to the presence of a motionless deer standing in the direct path of its headlights.


5 out of top 6 results are the expected semantically similar results. Acccuracy: 5/6 = 83%.



In [96]:
query_index(q2_embeddings.tolist())

The 16-year-old genius, having earned a PhD, contemplated global challenges and recognized science as the solution.
Having completed his doctorate at 16, the exceptionally gifted young prodigy pondered the difficulties confronting the world, intuitively perceiving science as the remedy.
Having finished his PhD at 16, the boy genius  contemplated the challenges the world faced, and intuited that science must be the solution.
The boy genius finished his PhD at 16 and believed science was the solution to the world's challenges.
After completing his PhD at the age of 16, the exceptionally talented young prodigy reflected on the global challenges and recognized science as the answer.
Finding sustainable energy solutions is crucial for a greener future.
Having successfully obtained his PhD at the remarkably young age of 16, the boy genius engaged in deep contemplation of the complex issues faced by the world. His intuition led him to firmly believe that science held the key to addressing the

5 out of top 6 results are the expected semantically similar results. Acccuracy: 5/6 = 83%.



In [97]:
delete_index('minilm-dotproduct')

In [98]:
pinecone.list_indexes()

[]

## Embeddings with `text-embedding-ada-002`

### Initialize openai

In [19]:
!pip install langchain openai tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
#config = dotenv_values(".env")
#openai_api_key = config['OPENAI_API_KEY'] # "get your token in http://hf.co/settings/tokens"

### Load or create dataset embeddings

In [35]:
text_embedding_model = "text-embedding-ada-002"
docs_file = f'{datadir}docs-{text_embedding_model}.pickle'

try:
  doc_embeddings = load(docs_file)

except:
  # if not present, create and save
  from langchain.embeddings import OpenAIEmbeddings
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
  doc_embeddings = embeddings.embed_documents(docs)
  save(doc_embeddings, docs_file)
  ""
print(f'shape: {len(doc_embeddings)}x{len(doc_embeddings[0])}')

shape: 57x1536


### Load or create query sentence embeddings

In [44]:
q1_file = f'{datadir}q1-{text_embedding_model}.pickle'
q2_file = f'{datadir}q2-{text_embedding_model}.pickle'

try:
  q1_embeddings = load(q1_file)
  q2_embeddings = load(q2_file)

except:
  # if not present, create and save
  from langchain.embeddings import OpenAIEmbeddings
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
  q1_embeddings = embeddings.embed_query(q1_sentence)
  save(q1_embeddings, q1_file)
  q2_embeddings = embeddings.embed_query(q2_sentence)
  save(q1_embeddings, q2_file)

print(f'size: {len(q1_embeddings)}')

size: 1536


In [45]:
docs_to_upsert = [(str(i), de) for i, de in enumerate(doc_embeddings)]

## Store embeddings in a vector database and perform similarity searches

### Cosine similarity search and results

In [39]:
index_name = 'ada-cosine'
create_index(index_name, len(doc_embeddings[0]), 'cosine')
populate_index(index_name, docs_to_upsert)

{'upserted_count': 57}

In [47]:
query_index(q1_embeddings)

The car skidded to stop for the deer that stood frozen in the headlights of the car.
The car skidded and stopped for the frozen deer in its headlights.
The car skidded to a standstill for the deer that remained motionless in the car's headlights.
The vehicle slid and came to a halt in response to the deer's immobility under the car's headlights.
The car skidded and stopped to avoid the motionless deer illuminated by its headlights.
As the car lost traction, it slid across the road and eventually halted abruptly, its brakes screeching, due to the presence of a motionless deer standing in the direct path of its headlights.
The deer cautiously crossed the road, looking out for oncoming vehicles.


6 out of top 6 results are the expected semantically similar results. Acccuracy: 6/6 = 100%.



In [48]:
query_index(q2_embeddings)

Having finished his PhD at 16, the boy genius  contemplated the challenges the world faced, and intuited that science must be the solution.
The 16-year-old genius, having earned a PhD, contemplated global challenges and recognized science as the solution.
The boy genius finished his PhD at 16 and believed science was the solution to the world's challenges.
Having completed his doctorate at 16, the exceptionally gifted young prodigy pondered the difficulties confronting the world, intuitively perceiving science as the remedy.
After completing his PhD at the age of 16, the exceptionally talented young prodigy reflected on the global challenges and recognized science as the answer.
Finding sustainable energy solutions is crucial for a greener future.
Access to clean water is a pressing global issue.


5 out of top 6 results are the expected semantically similar results. Acccuracy: 5/6 = 83%.



In [52]:
delete_index('ada-cosine')

### Euclidean Distance similarity search and results

In [54]:
index_name = 'ada-euclidean'
create_index(index_name, len(doc_embeddings[0]), 'euclidean')
populate_index(index_name, docs_to_upsert)

{'upserted_count': 57}

In [55]:
query_index(q1_embeddings)

The car skidded to stop for the deer that stood frozen in the headlights of the car.
The car skidded and stopped for the frozen deer in its headlights.
The car skidded to a standstill for the deer that remained motionless in the car's headlights.
The vehicle slid and came to a halt in response to the deer's immobility under the car's headlights.
The car skidded and stopped to avoid the motionless deer illuminated by its headlights.
As the car lost traction, it slid across the road and eventually halted abruptly, its brakes screeching, due to the presence of a motionless deer standing in the direct path of its headlights.
The deer cautiously crossed the road, looking out for oncoming vehicles.


6 out of top 6 results are the expected semantically similar results. Acccuracy: 6/6 = 100%.



In [56]:
query_index(q2_embeddings)

Having finished his PhD at 16, the boy genius  contemplated the challenges the world faced, and intuited that science must be the solution.
The 16-year-old genius, having earned a PhD, contemplated global challenges and recognized science as the solution.
The boy genius finished his PhD at 16 and believed science was the solution to the world's challenges.
Having completed his doctorate at 16, the exceptionally gifted young prodigy pondered the difficulties confronting the world, intuitively perceiving science as the remedy.
After completing his PhD at the age of 16, the exceptionally talented young prodigy reflected on the global challenges and recognized science as the answer.
Finding sustainable energy solutions is crucial for a greener future.
Access to clean water is a pressing global issue.


5 out of top 6 results are the expected semantically similar results. Acccuracy: 5/6 = 83%.



In [57]:
delete_index('ada-euclidean')